In [68]:
import networkx as nx
import time 
import pandas as pd
from utils.distance import get_distance
import matplotlib.pyplot as plt
import numpy as np
from utils.nn import nearest_neighbor
from utils.genalg import genetic_alg
from utils.simann import simulated_annealing
from data import locations

Graph = nx.Graph()

df = pd.read_csv("calculations/pass_distances.csv")

for a in df.itertuples():
    Graph.add_edge(a.Loc1, a.Loc2, weight=a._4)
    
results = []

In [69]:

# Nearest Neighbor
print("Running Nearest Neighbor")
start = time.time()
tour, distance = nearest_neighbor(Graph, start_index=list(Graph.nodes).index('Bulacan Provincial Capitol Malolos'))
nn_time = time.time() - start
print(f"NN Finished: {nn_time:.5f}s | Distance: {distance:.5f}m")

results.append(("Nearest Neighbor", nn_time, distance, tour))

Running Nearest Neighbor
NN Finished: 0.00019s | Distance: 110129.09446m


In [70]:
# Genetic Algorithm
print("Running Genetic Algorithm")
start = time.time()
tour, distance = genetic_alg(Graph, 100, 0.05, 1000)
ga_time = time.time() - start
print(f"GA Finished: {ga_time} | Distance: {distance}")
results.append(("Genetic Algorithm", ga_time, distance, tour))



Running Genetic Algorithm
GA Finished: 2.0941550731658936 | Distance: 86906.42196309914


In [71]:

# Simulated Annealing
print("Running Simulated Annealing")
start = time.time()
tour, distance = simulated_annealing(Graph, 10_000, 0.995,20_000)
sa_time = time.time() - start
print(f"SA Finished: {sa_time:.5f}s | Distance: {distance:.5f}m")
results.append(("Simulated Annealing", sa_time, distance, tour))


Running Simulated Annealing
SA Finished: 0.03998s | Distance: 96364.21307m


In [72]:
res_df = pd.DataFrame(results, columns=["Algorithm", "Time (s)", "Distance", "Tour"]).set_index("Algorithm")

In [73]:
res_df

,Time (s),Distance,Tour
Algorithm,,,
Nearest Neighbor,0.000189,110129.094459,"[Bulacan Provincial Capitol Malolos, Hiyas Bul..."
Genetic Algorithm,2.094155,86906.421963,"(Old PNR Guiguinto Station Guiguinto, Constan..."
Simulated Annealing,0.039976,96364.213074,"(Bulacan Provincial Capitol Malolos, Parish of..."


In [74]:
# Generate the Folium
import folium


for algo, time, distance, tour in res_df.itertuples():
    m = folium.Map(location=[locations[0].lat, locations[0].long], zoom_start=15)
    # Add marker to locations
    for loc in locations:
        folium.Marker((loc.lat, loc.long)).add_to(m)
    #Add edges 
    for i in range(len(tour)):
        loc1 = tour[i]
        loc2 = tour[(i + 1) % len(tour)]

        loc1 = next((l for l in locations if l.name == loc1))
        loc2 = next((l for l in locations if l.name == loc2))
        folium.PolyLine([(loc1.lat, loc1.long), (loc2.lat, loc2.long)], color="blue", weight=5, opacity=0.8).add_to(m)

    print(f"Saving file to {algo}.html...")
    m.save(f"calculations/{algo}.html")

Saving file to Nearest Neighbor.html...
Saving file to Genetic Algorithm.html...
Saving file to Simulated Annealing.html...
